In [291]:
import pandas as pd
import numpy as np

HH_PRICE = pd.read_excel(r"Z:\跨部門\螺絲報價\2 熱處理類別報價\螺絲熱處理牌價表-Gavin.xlsx").drop([0,1])
HH_PRICE = HH_PRICE.drop(HH_PRICE.columns[[4, 5, 7, 10, 11, 12]], axis = 1)
HH_PRICE.columns = ["Diameter(US)",
                    "Length_Range(US)",
                    "Diameter(EU)",
                    "Length_Range(EU)",
                    "H",  # DW, CB, Concrete
                    "H2", # SDS
                    "H3", # SDS-NO.5
                    "H1", # TP
                    "H6"  # Taptite thread
                   ]  # Match the number of columns
HH_PRICE["Length_Range(EU)"] = HH_PRICE["Length_Range(EU)"].str.replace("mm", "")
HH_PRICE["Length_Range(EU)"] = HH_PRICE["Length_Range(EU)"].str.replace("-", "~")
HH_PRICE["Diameter(EU)"] = HH_PRICE["Diameter(EU)"].str.replace("-", "~")
HH_PRICE

,Diameter(US),Length_Range(US),Diameter(EU),Length_Range(EU),H,H2,H3,H1,H6
2,#2~#4,"2""以下",M2.2~M2.9,50以下,7.7,7.7,x,7.7,7.7
3,NaN,NaN,M3,49以下,5,5.3/5.6(D040客人),6.6,6.1,6.6
4,NaN,NaN,NaN,50,6.1,6.1,6.6,6.4,6.6
5,NaN,NaN,NaN,51~60,8.1,8.1,x,NaN,NaN
6,NaN,NaN,M3.5,49以下,5,5.3/5.6(D040客人),6.6,6.1,6.6
...,...,...,...,...,...,...,...,...,...
66,NaN,NaN,NaN,滲碳層為心部+30條,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,M2.9以下損耗率 3%,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,M3.0以上損耗率 1.5%,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,夏季用電加價 +0.3/kg 5/15~10/15,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
def parse_interval(value):
    value = str(value).strip()  # Ensure it's a string and remove spaces

    if "以下" in value:  
        # Case: "40mm以下" → Convert to Interval(0, 40)
        try:
            end = int(value.replace("以下", "").strip())  # Extract number
            return pd.Interval(0, end, closed="right")  
        except ValueError:
            return None  # Handle unexpected format

    elif "~" in value:  
        # Case: "0~40" → Convert to Interval(0, 40)
        try:
            start, end = map(float, value.split("~"))
            return pd.Interval(start, end, closed="right")  
        except ValueError:
            return None  
    elif "以上" in value : 
        try:
            start = int(value.replace("以上", "").strip())  # Extract number
            return pd.Interval(start, np.inf, closed="right")  
        except ValueError:
            return None  # Handle unexpected format

    else:  
        # Case: "50" → Convert to Interval(50, 50)
        try:
            single_value = float(value)
            return pd.Interval(single_value, single_value, closed="both")  
        except ValueError:
            return None  
#HH_PRICE["Length_range(EU)"] = HH_PRICE["Length_range(EU)"].apply(parse_interval)
HH_PRICE["Diameter(EU)"] = HH_PRICE["Diameter(EU)"].ffill()  # forward filling 
HH_PRICE["Diameter(EU)"] = HH_PRICE["Diameter(EU)"].str.split("~")
HH_PRICE = HH_PRICE.explode('Diameter(EU)', ignore_index=True) 
HH_PRICE["Diameter(EU)"] = HH_PRICE["Diameter(EU)"].str.replace("M|以上", "", regex=True)
HH_PRICE["Length_Range(EU)"] = HH_PRICE["Length_Range(EU)"].apply(parse_interval)
HH_PRICE["Diameter(EU)"] = HH_PRICE["Diameter(EU)"].apply(parse_interval)
#x = get_heat_treatment_cost(HH_PRICE, "M3", 35, "H")+0.3

In [293]:
HH_PRICE

,Diameter(US),Length_Range(US),Diameter(EU),Length_Range(EU),H,H2,H3,H1,H6
0,#2~#4,"2""以下","[2.2, 2.2]","(0, 50]",7.7,7.7,x,7.7,7.7
1,#2~#4,"2""以下","[2.9, 2.9]","(0, 50]",7.7,7.7,x,7.7,7.7
2,NaN,NaN,"[3.0, 3.0]","(0, 49]",5,5.3/5.6(D040客人),6.6,6.1,6.6
3,NaN,NaN,"[3.0, 3.0]","[50.0, 50.0]",6.1,6.1,6.6,6.4,6.6
4,NaN,NaN,"[3.0, 3.0]","(51.0, 60.0]",8.1,8.1,x,NaN,NaN
...,...,...,...,...,...,...,...,...,...
72,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
74,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
75,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


In [294]:
HH_PRICE_I = HH_PRICE[:65].copy()
# handle the drill point cost, check D040 SDS  9X3"
def Process_SDS_HH(value):
    # find the value need to split 
    first_part = value.split("/") if "/" in str(value) else float(value)
    if type(first_part) == float:
        return first_part, first_part
    else:
        # this will contain the D040 & D092 ST price
        return float(first_part[0]), first_part[1].split("(")[0] if len(first_part[1]) > 5 else float(first_part[1]) 

HH_PRICE_I[["H2", "H2(EX)"]] = HH_PRICE_I["H2"].apply(Process_SDS_HH).apply(pd.Series)


In [295]:
#European Spec for HH price
def get_heat_treatment_cost_EU(HH_table, diameter, length, type_):
    row = HH_table[
        (HH_table["Diameter(EU)"].apply(lambda x: diameter in x if pd.notna(x) else False)) & 
        (HH_table["Length_Range(EU)"].apply(lambda x: length in x if pd.notna(x) else False))
    ]
    return float(row[type_].values[0]) if not row.empty else 0  # Return cost or 0 if no match
# summer peak demand
#x = get_heat_treatment_cost_EU(HH_PRICE_I, 3, 35, "H") + 0.3

In [315]:
# the price for USA spec
HH_USA = HH_PRICE_I.loc[HH_PRICE_I["Length_Range(US)"].notna(), ["Diameter(US)",
                                                 "Length_Range(US)",
                                                 "H",
                                                 "H2",
                                                 "H3",
                                                 "H1",
                                                 "H6",
                                                 "H2(EX)"]].copy().reset_index(drop = True)
HH_USA["Diameter(US)"] = HH_USA["Diameter(US)"].ffill().str.replace("#", "")
HH_USA

,Diameter(US),Length_Range(US),H,H2,H3,H1,H6,H2(EX)
0,2~4,"2""以下",7.7,7.7,x,7.7,7.7,7.7
1,2~4,"2""以下",7.7,7.7,x,7.7,7.7,7.7
2,6~7,"3""以下",5,5.3,6.6,6.1,6.6,5.6
3,6~7,"3-1/8""",5.4,6.1,9.3,6.8,X,6.1
4,8,"3""以下",5,5.3,6.6,6.1,6.6,5.6
5,8,"3-1/8""",5.4,6.1,9.3,6.8,X,6.1
6,8,"3-1/4""~3-1/2""",9.4,9.4,9.4,9.4,X,9.4
7,8,"3-5/8""~4""",14,14.0,x,NaN,NaN,14.0
8,9,"3""以下",5,5.3,6.6,6.1,6.6,5.6
9,9,"3-1/8""",5.4,6.1,9.3,6.8,X,6.1


In [316]:
def convert_fractional_inch(value):
    """Convert fractional inch size (e.g., '3-1/4"' -> 3.25) into a float."""
    value = value.replace('"', '')  # Remove inch symbol
    if "-" in value:  # Case like '3-1/4'
        whole, fraction = value.split("-")
        return float(whole) + float(Fraction(fraction))  # Convert fraction
    return float(value)

In [317]:
def parse_interval_inch(value):
    value = str(value).strip() # Ensure it's a string and remove spaces

    if "以下" in value:  
        # Case: "40mm以下" → Convert to Interval(0, 40)
        try:
            end = convert_fractional_inch(value.replace("以下", "")) # Extract number
            return pd.Interval(0, end, closed = "right")  
        except ValueError:
            return None  # Handle unexpected format

    elif "~" in value:  
        # Case: "0~40" → Convert to Interval(0, 40)
        try:
            start, end = map(convert_fractional_inch, value.split("~"))
            return pd.Interval(start, end, closed = "both")  
        except ValueError:
            return None  
    elif "以上" in value : 
        try:
            start = convert_fractional_inch(value.replace("以上", "").strip())  # Extract number
            return pd.Interval(start, np.inf, closed = "both")  
        except ValueError:
            return None  # Handle unexpected format

    else:  
        # Case: "50" → Convert to Interval(50, 50)
        try:
            single_value = convert_fractional_inch(value)
            return pd.Interval(single_value, single_value, closed = "both")  
        except ValueError:
            return None

In [318]:
HH_USA["Length_Range(US)"] = HH_USA["Length_Range(US)"].apply(parse_interval_inch)
HH_USA["Diameter(US)"] = HH_USA["Diameter(US)"].apply(parse_interval_inch)
HH_USA

,Diameter(US),Length_Range(US),H,H2,H3,H1,H6,H2(EX)
0,"[2.0, 4.0]","(0, 2.0]",7.7,7.7,x,7.7,7.7,7.7
1,"[2.0, 4.0]","(0, 2.0]",7.7,7.7,x,7.7,7.7,7.7
2,"[6.0, 7.0]","(0, 3.0]",5,5.3,6.6,6.1,6.6,5.6
3,"[6.0, 7.0]","[3.125, 3.125]",5.4,6.1,9.3,6.8,X,6.1
4,"[8.0, 8.0]","(0, 3.0]",5,5.3,6.6,6.1,6.6,5.6
5,"[8.0, 8.0]","[3.125, 3.125]",5.4,6.1,9.3,6.8,X,6.1
6,"[8.0, 8.0]","[3.25, 3.5]",9.4,9.4,9.4,9.4,X,9.4
7,"[8.0, 8.0]","[3.625, 4.0]",14,14.0,x,NaN,NaN,14.0
8,"[9.0, 9.0]","(0, 3.0]",5,5.3,6.6,6.1,6.6,5.6
9,"[9.0, 9.0]","[3.125, 3.125]",5.4,6.1,9.3,6.8,X,6.1


In [320]:
def get_heat_treatment_cost_USA(HH_table, diameter, length, type_):
    row = HH_table[
        (HH_table["Diameter(US)"].apply(lambda x: diameter in x if pd.notna(x) else False)) & 
        (HH_table["Length_Range(US)"].apply(lambda x: length in x if pd.notna(x) else False))
    ]
    return float(row[type_].values[0]) if not row.empty else 0  # Return cost or 0 if no match
x = get_heat_treatment_cost_USA(HH_USA, 6, Fraction(5, 4), "H") + 0.3
x

5.3